In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0, '..//src/')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import main
from pathlib import Path


sns.set_palette(sns.color_palette("colorblind", 15))
plt.rcParams['font.family'] = 'Times New Roman'
font_size = 12
params = {'axes.labelsize': font_size+2,
          'axes.titlesize':font_size+4,
          'legend.fontsize': font_size, 
          'xtick.labelsize': font_size,
          'ytick.labelsize': font_size,
          'font.size': font_size}
plt.rcParams.update(params)
sns.set_style("whitegrid")
fig_format = "pdf"
dpi = 300
figsize = (12, 3.5)
figsize_map = (16, 8)
sns.palplot(sns.color_palette())

In [176]:
PATH_ONEDRIVE = Path(r'D:\Profile data Cardiff\OneDrive - Cardiff University')
path_saving_results = PATH_ONEDRIVE / r'04 - Projects\25 - Geothermal district heating\04 - Data\viable_lsoas'
path_scotland_results = PATH_ONEDRIVE / r'04 - Projects\22 - Heat demand scotland\data'
# path_ukerc_england_wales_df = Path(r"C:\Users\sceac10\Cardiff University\National Grid Project - Documents\General\Paper\data for UKERC")
path_england_wales_df = PATH_ONEDRIVE / r'04 - Projects\03 - PhD\03 - Analysis\03 - LSOAs\00 - Data'
ukerc_save_data =PATH_ONEDRIVE / r'04 - Projects\20 - UKERC\03 - Code\flexibilitydwellings\data\UKERC\01 - Thermal_Characteristics'

path_scotland_la_region_lookup = PATH_ONEDRIVE / r'04 - Projects\00 - Final data\Geo_lookup_tables\Scotland_LAregionlookup.csv'

path_gb_data = PATH_ONEDRIVE / r'04 - Projects\00 - Final data\Annual_demand'

In [ ]:
# thermal characteristics LSOA data
lookup_geolocation_df = pd.read_csv(ukerc_save_data/f"Thermal_characteristics_afterEE.csv", index_col=0)
lookup_geolocation_df.head()


In [178]:

lookup_lsoa_LA = dict(zip(lookup_geolocation_df['LSOA_code'].values, lookup_geolocation_df['Local Authority'].values))
lookup_lsoa_Region = dict(zip(lookup_geolocation_df['LSOA_code'].values, lookup_geolocation_df['Region'].values))


In [ ]:
scenario = 'before'
england_wales_df = pd.read_csv(path_england_wales_df/f"LSOAs_in_England_Wales_{scenario}_EE_heat_demand.csv")
england_wales_df.columns = [x.lower() for x in england_wales_df.columns]
england_wales_df.head()

In [ ]:
england_wales_df.columns = [x+" (m2)" if "average floor area" in x else x for x in england_wales_df.columns ]
for x in england_wales_df.columns:
  print(x)

In [ ]:
scotland_df = pd.read_csv(path_scotland_results/"full_scotland_results.csv", index_col=0)
scotland_df.columns = [x.lower() for x in scotland_df.columns]
scotland_df.columns = [c.replace('resistance', 'resistance heating') for c in scotland_df.columns]
scotland_df.columns = [c.replace('solid fuel boiler', 'biomass boiler') for c in scotland_df.columns]
scotland_df.head()

In [182]:
lookup_lsoa_LA_scotland = dict(scotland_df[['lsoa11cd', 'ladnm']].values)

lookup_la_region_scotland = pd.read_csv(path_scotland_la_region_lookup, index_col=0)
lookup_la_region_scotland = dict(lookup_la_region_scotland.values)

In [183]:
gb_df = pd.concat([scotland_df, england_wales_df], axis=0)#.dropna(axis=1)

In [ ]:
cols_to_keep = ['lsoa11cd', 'msoa11cd', 'road length (m)', 'area (km2)']
cols_to_keep = cols_to_keep + [x for x in gb_df.columns if f"average heat demand {scenario} energy efficiency measures for" in x]
cols_to_keep = cols_to_keep + [x for x in gb_df.columns if "average floor area" in x]
cols_to_keep = cols_to_keep + [x for x in gb_df.columns if "number of" in x and 'in 2018' in x and 'total' not in x]
cols_to_keep = cols_to_keep + [x for x in gb_df.columns if "number of" in x and 'in 2011' in x and 'total' not in x]
cols_to_keep

In [ ]:
gb_df = gb_df[cols_to_keep]
gb_df['Region'] = gb_df['lsoa11cd'].map(lookup_lsoa_Region)
gb_df['Local Authority'] = gb_df['lsoa11cd'].map(lookup_lsoa_LA)
gb_df['Local Authority'] = gb_df['Local Authority'].fillna(gb_df['lsoa11cd'].map(lookup_lsoa_LA_scotland))
gb_df['Region'] = gb_df['Region'].fillna(gb_df['Local Authority'].map(lookup_la_region_scotland))
gb_df.head()

In [186]:
floor_area_cols = [x for x in gb_df.columns if "average floor area" in x]
gb_df[floor_area_cols] = gb_df[floor_area_cols].fillna(gb_df[floor_area_cols].mean())

In [187]:
gb_df.columns = [x.capitalize().replace('kwh', 'kWh') for x in gb_df.columns]

In [ ]:
gb_df.head()

In [189]:
for c in gb_df.columns:
  if len(gb_df[gb_df[c].isna()]) > 0:
    print(f'{c} for {len(gb_df[gb_df[c].isna()])}')

In [190]:
gb_df.reset_index(inplace=True, drop=True)
gb_df.to_csv(path_gb_data/f"GB_Annual_heat_demand_{scenario}_EE_LSOA.csv")